In [ ]:
import random
import os
#os.environ['TRIPLE_STORE_TYPE'] = 'virtuoso'
from plastering.metadata_interface import *
from plastering.inferencers.zodiac import ZodiacInterface
#from plastering.inferencers.hong_al import HongAlInterface

In [ ]:
target_building = 'ebu3b'
source_buildings = ['ap_m', 'uva_cse', 'ebu3b']
#source_nums = [200, 200, 10]
target_srcids = LabeledMetadata.objects(building=target_building).distinct('srcid')

In [ ]:
# TODO: Automatically download Brick.ttl form the URL.

## Supervised Learning Evaluation
We randomly select training exmamples and evalute the performance.

In [ ]:
zodiac = ZodiacInterface(target_building,
                         target_srcids,
                         source_buildings,
                        )
inferencers = [zodiac]
training_srcids = random.sample(target_srcids, 100)
test_srcids = [srcid for srcid in target_srcids if srcid not in training_srcids]
zodiac.update_model(training_srcids)
metrics = zodiac.evaluate(test_srcids)

In [ ]:
print(metrics)

## Active Learning Evaluation
We will choose training examples in each stage based on each algorithm's implementation.

In [ ]:
TOTAL_ITER_NUM = 200
NEW_SAMPLE_NUM = 1
zodiac = ZodiacInterface(target_building,
                         target_srcids,
                         source_buildings,
                        )
for i in range(0, TOTAL_ITER_NUM):
    new_srcids = zodiac.select_informative_samples(NEW_SAMPLE_NUM)
    zodiac.update_model(new_srcids)
    metrics = zodiac.evaluate(target_srcids)
    print(metrics)

### Shortcut
It is common to run active learning in multiple iterations so we recommend to implement ``learn_auto()`` that learns multiple stages with active learning. It is basically a wrapper of the above implementation.

In [ ]:
zodiac.learn_auto()